In [ ]:
import random
from glob import glob
from pathlib import Path
import cv2
import numpy as np

LABELS_DIR = "labels"
BACKGROUNDS_DIR = "backgrounds"
OUT_IMG_DIR = Path("output/images")
OUT_ANN_DIR = Path("output/labels")

NUM_SYNTHETIC      = 1000
OUTPUT_W, OUTPUT_H = 640, 480

MARGIN             = 50
MIN_SCALE          = 0.1
MAX_SCALE_DEFAULT  = 0.4
DX_DY_RANGE        = 20
NOISE_SIGMA        = 5

MIN_LABELS_PER_IMG = 2
MAX_LABELS_PER_IMG = 4
MAX_PLACE_ATTEMPTS = 30
MAX_JITTER_ATTEMPTS= 60

GAMMA_PROB  = 0.6
SHADOW_PROB = 0.5

DEBUG_DRAW  = False
DEBUG_LIMIT = 20


HASH_MAP = {
    '105_145x30': 0,
    '105_50x80' : 1,
    '106_100x30': 2,
    '106_50x80' : 3,
    '107_50x80' : 4,
    '108_50x80' : 5,
    '409_146x30': 6,
    '409_50x80' : 7,
    '410_50x80' : 8,
    '411_50x80' : 9,
    '412_50x80' : 10,
    '417_50x80' : 11,
}

OUT_IMG_DIR.mkdir(parents=True, exist_ok=True)
OUT_ANN_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
def gamma_correction(img:np.ndarray)->np.ndarray:
    if random.random()>GAMMA_PROB:
        return img
    gamma=random.uniform(0.6,1.4)
    inv=1.0/gamma
    table=np.array([(i/255.0)**inv*255 for i in range(256)],dtype=np.uint8)
    return cv2.LUT(img,table)

def add_shadow(img:np.ndarray)->np.ndarray:
    if random.random()>SHADOW_PROB:
        return img
    h,w=img.shape[:2]
    mask=np.zeros((h,w),np.float32)
    if random.random()<0.5:
        n_pts = random.randint(3, 8)
        pts = np.column_stack((
            np.random.randint(0, w, n_pts),
            np.random.randint(0, h, n_pts)
        )).astype(np.int32)
        cv2.fillPoly(mask,[pts],1.0)
    else:
        center=(random.randint(0,w),random.randint(0,h))
        axes=(random.randint(w//4,w),random.randint(h//4,h))
        angle=random.uniform(0,360)
        cv2.ellipse(mask,center,axes,angle,0,360,1.0,-1)
    k=random.randrange(101,201,2)
    mask=cv2.GaussianBlur(mask,(k,k),0)
    intensity=random.uniform(0.3,0.7)
    return (img.astype(np.float32)*(1-intensity*mask[...,None])).astype(np.uint8)

def add_noise_jpeg(img:np.ndarray)->np.ndarray:
    noise=np.random.normal(0,NOISE_SIGMA,img.shape).astype(np.int16)
    img=cv2.add(img,noise,dtype=cv2.CV_8U)
    _,enc=cv2.imencode('.jpg',img,[cv2.IMWRITE_JPEG_QUALITY,random.randint(60,95)])
    return cv2.imdecode(enc,cv2.IMREAD_COLOR)

def order_corners(pts:np.ndarray)->np.ndarray:
    y_sorted=pts[np.argsort(pts[:,1])]
    tl,tr=y_sorted[:2][np.argsort(y_sorted[:2,0])]
    bl,br=y_sorted[2:][np.argsort(y_sorted[2:,0])]
    return np.array([tl,tr,br,bl],np.float32)

def overlap(r1,r2):
    return not (r1[2]<=r2[0] or r2[2]<=r1[0] or r1[3]<=r2[1] or r2[3]<=r1[1])

In [ ]:
label_infos = []
for p in sorted(glob(f"{LABELS_DIR}/*")):
    img = cv2.imread(p, cv2.IMREAD_UNCHANGED)
    if img is None:
        continue
    alpha = img[:,:,3] if img.shape[2]==4 else None
    mask = (alpha>10) if alpha is not None else (cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)>10)
    ys,xs = np.where(mask)
    if ys.size==0:
        continue
    x1,x2,y1,y2 = xs.min(),xs.max(),ys.min(),ys.max()
    trimmed = img[y1:y2+1,x1:x2+1].copy()
    label_infos.append({"img":trimmed,"w":trimmed.shape[1],"h":trimmed.shape[0],"name":Path(p).stem})

if not label_infos:
    raise RuntimeError("Разметка не найдена!")

In [ ]:
CLASS_ID_MAP = HASH_MAP.copy()
next_id = max(CLASS_ID_MAP.values(),default=-1)+1
for info in label_infos:
    if info["name"] not in CLASS_ID_MAP:
        CLASS_ID_MAP[info["name"]]=next_id; next_id+=1

backgrounds = [cv2.imread(p,cv2.IMREAD_COLOR) for p in sorted(glob(f"{BACKGROUNDS_DIR}/*"))]
if not backgrounds:
    raise RuntimeError("No backgrounds found")

_BORDER_TRANSPARENT = getattr(cv2,"BORDER_TRANSPARENT",cv2.BORDER_CONSTANT)

In [ ]:
counter=attempts=0
while counter<NUM_SYNTHETIC and attempts<NUM_SYNTHETIC*4:
    attempts+=1
    bg=random.choice(backgrounds).copy()
    h_bg0,w_bg0=bg.shape[:2]

    objs=[]; placed=[]
    for _ in range(random.randint(MIN_LABELS_PER_IMG,MAX_LABELS_PER_IMG)):
        placed_flag=False
        for _ in range(MAX_PLACE_ATTEMPTS):
            info=random.choice(label_infos)
            lbl_o,lw,lh,name=info["img"],info["w"],info["h"],info["name"]
            limit_scale=min((w_bg0-2*MARGIN)/lw,(h_bg0-2*MARGIN)/lh)
            max_scale=min(limit_scale,MAX_SCALE_DEFAULT)
            if max_scale<MIN_SCALE: break
            scale=random.uniform(MIN_SCALE,max_scale)
            nw,nh=max(1,int(lw*scale)),max(1,int(lh*scale))
            lbl=cv2.resize(lbl_o,(nw,nh),interpolation=cv2.INTER_AREA)
            bx=random.uniform(MARGIN,w_bg0-MARGIN-nw)
            by=random.uniform(MARGIN,h_bg0-MARGIN-nh)
            src=np.float32([[0,0],[nw,0],[nw,nh],[0,nh]])
            dst_base=src.copy(); dst_base[:,0]+=bx; dst_base[:,1]+=by
            for _ in range(MAX_JITTER_ATTEMPTS):
                dst=(dst_base+np.random.uniform(-DX_DY_RANGE,DX_DY_RANGE,(4,2))).astype(np.float32)
                if (dst[:,0]>=0).all() and (dst[:,0]<=w_bg0).all() and (dst[:,1]>=0).all() and (dst[:,1]<=h_bg0).all():
                    break
            else: continue
            if any(overlap((dst[:,0].min(),dst[:,1].min(),dst[:,0].max(),dst[:,1].max()),r) for r in placed):
                continue
            M=cv2.getPerspectiveTransform(src,dst)
            warp=cv2.warpPerspective(lbl,M,(w_bg0,h_bg0),borderMode=_BORDER_TRANSPARENT)
            alpha=(warp[:,:,3] if warp.shape[2]==4 else cv2.cvtColor(warp,cv2.COLOR_BGR2GRAY))>10
            if alpha.sum()==0: continue
            mask_f=cv2.GaussianBlur(alpha.astype(float),(15,15),0)[...,None]
            bg=(bg*(1-mask_f)+warp[:,:,:3]*mask_f).astype(np.uint8)
            kp_px=order_corners(dst.copy())
            x1,y1,x2,y2=kp_px[:,0].min(),kp_px[:,1].min(),kp_px[:,0].max(),kp_px[:,1].max()
            placed.append((x1,y1,x2,y2))
            objs.append({"kp":kp_px,"bbox":np.array([x1,y1,x2,y2],np.float32),"class":CLASS_ID_MAP[name]})
            placed_flag=True; break
        if not placed_flag: continue

    if not objs: continue


    bg=gamma_correction(bg)
    bg=add_shadow(bg)
    bg=add_noise_jpeg(bg)

    sx,sy=OUTPUT_W/w_bg0,OUTPUT_H/h_bg0
    bg=cv2.resize(bg,(OUTPUT_W,OUTPUT_H),interpolation=cv2.INTER_AREA)
    if DEBUG_DRAW and counter<DEBUG_LIMIT: dbg=bg.copy()

    with open(OUT_ANN_DIR/f"image_{counter:05d}.txt","w",encoding="utf-8") as fa:
        for o in objs:
            o["kp"][:,0]*=sx; o["kp"][:,1]*=sy
            o["bbox"][0::2]*=sx; o["bbox"][1::2]*=sy
            kp=o["kp"].copy(); kp[:,0]/=OUTPUT_W; kp[:,1]/=OUTPUT_H
            x1,y1,x2,y2=o["bbox"]/[OUTPUT_W,OUTPUT_H,OUTPUT_W,OUTPUT_H]
            xc,yc=(x1+x2)/2,(y1+y2)/2; bw,bh=x2-x1,y2-y1
            nums=[o["class"],xc,yc,bw,bh]+kp.flatten().tolist()
            fa.write(" ".join(f"{v:.6f}" for v in nums)+"\n")
            if DEBUG_DRAW and counter<DEBUG_LIMIT:
                cv2.rectangle(dbg,(int(x1*OUTPUT_W),int(y1*OUTPUT_H)),(int(x2*OUTPUT_W),int(y2*OUTPUT_H)),(0,255,255),2)
                for kx,ky in kp:
                    cv2.circle(dbg,(int(kx*OUTPUT_W),int(ky*OUTPUT_H)),5,(0,255,0),-1)

    cv2.imwrite(str(OUT_IMG_DIR/f"image_{counter:05d}.jpg"),bg)
    if DEBUG_DRAW and counter<DEBUG_LIMIT:
        cv2.imwrite(str(OUT_IMG_DIR/f"debug_{counter:05d}.jpg"),dbg)

    counter+=1

print(f"✔ Generated {counter} images ({OUTPUT_W}×{OUTPUT_H}). Annotations stored in '{OUT_ANN_DIR}'.")